# Preços de Medicamentos - Consumidor
### Nesse projeto irei criar uma API para saber de fato de quanto foi o aumento no preço dos médicamento nesse último reajuste no brasil.

<img src="https://i.imgur.com/W7DMmnl.jpeg" />

Os dados representam a lista de preços de Medicamentos , contemplando o preço Fábrica, ou preço **fabricante (PF)**, que é o preço máximo que pode ser praticado pelas empresas produtoras ou importadoras do produto e pelas empresas distribuidoras. O PF indica o preço máximo permitido para venda a farmácias e drogarias e o **Preço Máximo ao Consumidor (PMC)** indica o preço teto de venda ao consumidor.

Fonte dos dados: https://dados.anvisa.gov.br/dados

### Agência Nacional de Vigilância Sanitária - ANVISA

Criada pela Lei nº 9.782, de 26 de janeiro 1999, a Agência Nacional de Vigilância Sanitária (Anvisa) é uma autarquia sob regime especial, que tem sede e foro no Distrito Federal, e está presente em todo o território nacional por meio das coordenações de portos, aeroportos, fronteiras e recintos alfandegados.

Tem por finalidade institucional promover a proteção da saúde da população, por intermédio do controle sanitário da produção e consumo de produtos e serviços submetidos à vigilância sanitária, inclusive dos ambientes, dos processos, dos insumos e das tecnologias a eles relacionados, bem como o controle de portos, aeroportos, fronteiras e recintos alfandegados.

- Data da criação do banco de dados: 31 de maio de 2017
- Última atualização: 13 de outubro de 2020 para o DataFrame anterior
- Reajuste: 08 de fevereiro de 2021 para o DataFrame reajuste

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline



import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 999)

In [4]:
import mysql.connector
import pandas as a

conn = mysql.connector.connect(host="localhost", user="root", passwd="1234", db="preco_medicamentos_brasil")
anterior = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_10_2020" 
                      ,conn)


reajuste = a.read_sql_query("SELECT * FROM TA_PRECO_MEDICAMENTO_03_2021" 
                      ,conn)

In [5]:
anterior.head()

,SUBSTANCIA,CNPJ,LABORATORIO,CODIGO GGREM,REGISTRO,EAN 1,EAN 2,EAN 3,PRODUTO,APRESENTACAO,CLASSE TERAPEUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),REGIME DE PRECO,PF Sem Impostos,PF 0%,PF 12%,PF 17%,PF 17% ALC,"PF 17,5%","PF 17,5% ALC",PF 18%,PF 18% ALC,PF 20%,PMC 0%,PMC 12%,PMC 17%,PMC 17% ALC,"PMC 17,5%","PMC 17,5% ALC",PMC 18%,PMC 18% ALC,PMC 20%,RESTRICAO HOSPITALAR,CAP,CONFAZ 87,ICMS 0%,ANALISE RECURSAL,LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS),COMERCIALIZACAO 2019,TARJA
0,ACETATO DE DEXAMETASONA;CLOTRIMAZOL,18.459.628/0001-15,BAYER S.A.,"5,39E+14","1,71E+12","7,89E+12",-,-,BAYCUTEN N,"10 MG/G + 0,443 MG/G CREM DERM CT BG AL X 40 G",D7B2 - CORTICOESTERÓIDES ASSOCIADOS A ANTIMICO...,Novo,Regulado,"19,74","22,11","25,55","27,32","23,78","27,51","23,93","27,71","27,71","28,51","29,66","34,13","36,42","32,87","36,66","33,08","36,92","33,28","37,95",Năo,Năo,Năo,Năo,,Negativa,Sim,Tarja Vermelha
1,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 1 SER PREENCHIDA,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1228,49","1228,49","1396,01","1480,11","1480,11","1489,08","1489,08","1498,16","1498,16","1535,61","1698,32","1929,9","2046,17","2046,17","2058,57","2058,57","2071,12","2071,12","2122,89",Năo,Năo,Năo,Năo,,Positiva,Năo,Tarja Vermelha
2,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,125 MG/ML SOL INJ CT 4 SER PREENC VD TRANS + D...,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"4914,04","4914,04","5584,14","5920,53","5920,53","5956,41","5956,41","5992,73","5992,73","6142,55","6793,37","7719,75","8184,79","8184,79","8234,39","8234,39","8284,6","8284,6","8491,72",Năo,Năo,Sim,Năo,,Positiva,Sim,Tarja Vermelha (*)
3,ABATACEPTE,56.998.982/0001-07,BRISTOL-MYERS SQUIBB FARMACĘUTICA LTDA,"5,05E+14","1,02E+12","7,90E+12",-,-,ORENCIA,250 MG PO LIOF INJ 1 CT 1 FA + SER DESCARTÁVEL,M1C - AGENTES ANTI-REUMÁTICOS ESPECÍFICOS,Biológico,Regulado,"1547,45","1547,45","1758,47","1864,4","1864,4","1875,7","1875,7","1887,13","1887,13","1934,31",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha
4,ABCIXIMABE,51.780.468/0001-87,JANSSEN-CILAG FARMACĘUTICA LTDA,"5,15E+14","1,12E+12","7,90E+12",-,-,REOPRO,2 MG/ML SOL INJ CT FA VD INC X 5 ML,"B1C3 - INIBIDORES DA AGREGAÇĂO PLAQUETÁRIA, AN...",-,Regulado,"2003,06","2003,06","2276,21","2413,32","2413,32","2427,95","2427,95","2442,76","2442,76","2503,83",,,,,,,,,,Sim,Năo,Năo,Năo,,Positiva,Sim,Tarja Vermelha (*)


In [6]:
reajuste.head()

,SUBSTANCIA (REAJUSTE),CNPJ (REAJUSTE),LABORATORIO (REAJUSTE),CODIGO GGREM (REAJUSTE),REGISTRO (REAJUSTE),EAN 1 (REAJUSTE),EAN 2 (REAJUSTE),EAN 3 (REAJUSTE),PRODUTO (REAJUSTE),APRESENTACAO (REAJUSTE),CLASSE TERAPEUTICA (REAJUSTE),TIPO DE PRODUTO (STATUS DO PRODUTO) (REAJUSTE),REGIME DE PRECO (REAJUSTE),PF Sem Impostos (REAJUSTE),PF 0% (REAJUSTE),PF 12% (REAJUSTE),PF 17% (REAJUSTE),PF 17% ALC (REAJUSTE),"PF 17,5% (REAJUSTE)","PF 17,5% ALC (REAJUSTE)",PF 18% (REAJUSTE),PF 18% ALC (REAJUSTE),PF 20% (REAJUSTE),PMC 0% (REAJUSTE),PMC 12% (REAJUSTE),PMC 17% (REAJUSTE),PMC 17% ALC (REAJUSTE),"PMC 17,5% (REAJUSTE)","PMC 17,5% ALC (REAJUSTE)",PMC 18% (REAJUSTE),PMC 18% ALC (REAJUSTE),PMC 20% (REAJUSTE),RESTRICAO HOSPITALAR (REAJUSTE),CAP (REAJUSTE),CONFAZ 87 (REAJUSTE),ICMS 0% (REAJUSTE),ANALISE RECURSAL (REAJUSTE),LISTA DE CONCESSAO DE CREDITO TRIBUTARIO (PIS/COFINS) (REAJUSTE),COMERCIALIZACAO 2019 (REAJUSTE),TARJA (REAJUSTE)
0,ACETATO DE LANREOTIDA,07.718.721/0001-80,BEAUFOUR IPSEN FARMACÊUTICA LTDA,"5,39E+14","1,70E+12","7,90E+12","7,90E+12",-,SOMATULINE AUTOGEL,120 MG SOL INJ LIB PROL CT ENV AL/PLAS SER PRE...,H1C2 - HORMÔNIOS ANTICRESCIMENTO,Novo,Regulado,"2456,35","2456,35","2791,31","2959,46","2959,46","2977,39","2977,39","2995,55","2995,55","3070,44","3395,76","3858,82","4091,28","4091,28","4116,07","4116,07","4141,17","4141,17","4244,7",Não,Sim,Sim,Não,,Positiva,Não,Tarja -(*)
1,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 20,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"57,69","57,69","65,56","69,51","69,51","69,93","69,93","70,35","70,35","72,11","79,75","90,63","96,09","96,09","96,67","96,67","97,25","97,25","99,69",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
2,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 30,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"86,54","86,54","98,34","104,27","104,27","104,9","104,9","105,54","105,54","108,18","119,64","135,95","144,15","144,15","145,02","145,02","145,9","145,9","149,55",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
3,ACICLOVIR,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,"5,26E+14","1,06E+12","7,90E+12",-,-,ACICLOVIR,400 MG COM CT BL AL PLAS PVDC OPC X 50,J5B3 - ANTIVIRAIS PARA HERPES,Genérico,Regulado,"144,23","144,23","163,9","173,77","173,77","174,82","174,82","175,89","175,89","180,29","199,39","226,58","240,23","240,23","241,68","241,68","243,16","243,16","249,24",Não,Não,Não,Não,,Positiva,Não,Tarja Vermelha(*)
4,ACTAEA RACEMOSA L.,78.950.011/0001-20,HERBARIUM LABORATORIO BOTANICO LTDA,"5,11E+14","1,19E+12","7,90E+12",-,-,CLIFEMIN,160 MG COM REV CT BL AL PLAS PVDC TRANS X 15,G3J - MODULADORES SELETIVOS DO RECEPTOR DE EST...,Fitoterápico,Liberado,"34,1","38,19","44,13","47,19","41,08","47,52","41,33","47,86","41,58","49,24","51,23","58,95","62,9","56,79","63,33","57,14","63,76","57,48","65,54",Não,Não,Não,Não,,Negativa,Não,Tarja -(*)
